In [1]:
import torch
from torch import optim
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
# from torchvision.transforms import ToTensorfrom 
from torch.utils.tensorboard import SummaryWriter

import matplotlib.pyplot as plt
import numpy as np
import os

writer = SummaryWriter()
writer.close()

In [2]:
class PointCloudDataset(Dataset):
    def __init__(self, pts_file):
        points = np.loadtxt(pts_file, delimiter=' ')
        # do i need to min max intensity, return number, number of returns, etc.? probably not
        for i in range(3):
            dim_min, dim_max = min(points[:,i]), max(points[:,i])
            points[:,i] = (points[:,i] - dim_min) / (dim_max - dim_min)
        self.data = points
        
    def __len__(self):
        return len(self.data) // 5000
        
    def __getitem__(self, idx):
        # Return batches of 5000 points
        xyzirn = self.data[idx * 5000: (idx + 1) * 5000, :6]  # x, y, z, intensity, return number, number of returns
        label = self.data[idx * 5000: (idx + 1) * 5000, 6] == 8

        xyzirn = torch.from_numpy(xyzirn.T).float()
        label = torch.tensor(label).long()
        
        return xyzirn, label

training_data = PointCloudDataset(r"C:\Users\and13375\Documents\3D Point Segmentation\Test\Vaihingen\3DLabeling\Vaihingen3D_Traininig.pts")
testing_data = PointCloudDataset(r"C:\Users\and13375\Documents\3D Point Segmentation\Test\Vaihingen\3DLabeling\Vaihingen3D_EVAL_WITH_REF.pts")
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True) # can/should i use shuffle, try lowering it?
test_dataloader = DataLoader(testing_data, batch_size=64, shuffle=False)

num_classes = 2

In [3]:
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

Feature batch shape: torch.Size([64, 6, 5000])
Labels batch shape: torch.Size([64, 5000])


In [4]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [5]:
class STNkd(nn.Module):
    def __init__(self, k=64):
        super(STNkd, self).__init__()
        self.mlp1 = nn.Sequential(torch.nn.Conv1d(k, 64, 1), nn.BatchNorm1d(64), nn.GELU())
        self.mlp2 = nn.Sequential(torch.nn.Conv1d(64, 128, 1), nn.BatchNorm1d(128), nn.GELU())
        self.mlp3 = nn.Sequential(torch.nn.Conv1d(128, 1024, 1), nn.BatchNorm1d(1024), nn.GELU())
        self.mlp4 = nn.Sequential(nn.Linear(1024, 512), nn.BatchNorm1d(512), nn.GELU())
        self.mlp5 = nn.Sequential(nn.Linear(512, 256), nn.BatchNorm1d(256), nn.GELU())
        self.fc = nn.Linear(256, k*k)

        self.k = k

    def forward(self, x):
        batchsize = x.size()[0]
        x = self.mlp1(x)
        x = self.mlp2(x)
        x = self.mlp3(x)

        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)

        x = self.mlp4(x)
        x = self.mlp5(x)
        x = self.fc(x)

        iden = torch.eye(self.k, requires_grad=True).repeat(batchsize,1,1)
        if x.is_cuda:
            iden = iden.cuda()
        x = x.view(-1, self.k, self.k) + iden

        return x

In [6]:
class PoinNet(nn.Module):
    def __init__(self, input_dim=6, num_classes=2):
        super().__init__()

        # Should GELU be after LayerNorm?
        self.stn1 = STNkd(k=input_dim)
        self.mlp1 = nn.Sequential(nn.Conv1d(input_dim, 64, kernel_size=1), nn.BatchNorm1d(64), nn.GELU())
        self.mlp2 = nn.Sequential(nn.Conv1d(64, 64, kernel_size=1), nn.BatchNorm1d(64), nn.GELU())
        
        self.stn2 = STNkd(k=64)
        self.mlp3 = nn.Sequential(nn.Conv1d(64, 64, kernel_size=1), nn.BatchNorm1d(64), nn.GELU())
        self.mlp4 = nn.Sequential(nn.Conv1d(64, 128, kernel_size=1), nn.BatchNorm1d(128), nn.GELU())
        self.mlp5 = nn.Sequential(nn.Conv1d(128, 1024, kernel_size=1), nn.BatchNorm1d(1024), nn.GELU())
        
        self.mlp6 = nn.Sequential(nn.Linear(1088, 512), nn.LayerNorm(512), nn.GELU())
        self.mlp7 = nn.Sequential(nn.Linear(512, 256), nn.LayerNorm(256), nn.GELU())
        self.mlp8 = nn.Sequential(nn.Linear(256, 128), nn.LayerNorm(128), nn.GELU())
        self.fc = nn.Linear(128, num_classes)

        self.debug = nn.Conv1d(input_dim, num_classes, 1)

    def forward(self, x):
        n_pts = x.size()[2]

        trans6x6 = self.stn1(x) 
        x = x.transpose(2, 1)
        x = torch.bmm(x, trans6x6)
        x = x.transpose(2, 1)

        x = self.mlp1(x)
        # print(1, x.size())
        x = self.mlp2(x)
        # print(2, x.size())
        
        
        trans64x64 = self.stn2(x)
        x = x.transpose(2, 1)
        x = torch.bmm(x, trans64x64)
        local_features = x.transpose(2, 1)
        
        x = self.mlp3(local_features)
        # print(3, x.size())
        x = self.mlp4(x)
        print(4, x.size())
        x = self.mlp5(x)
        print(5, x.size())
        x = torch.max(x, 2)[0]
        print(6, x.size())
        
        # FOR CLASSIFICATION
        # x = self.mlp6(x)
        # print(6, x.size())
        # x = self.mlp7(x)
        # print(7, x.size())
        # x = self.fc(x)
        # print(8, x.size())
        # return F.log_softmax(x, dim=1)

        global_features = x.unsqueeze(2).repeat(1, 1, n_pts)
        print(7, global_features.size())
        x = torch.cat([local_features, global_features], 1)
        print(8, x.size())

        x = x.transpose(2, 1)
        x = self.mlp6(x)
        print(9, x.size())
        x = self.mlp7(x)
        print(10, x.size())
        x = self.mlp8(x)
        print(11, x.size())
        x = self.fc(x)
        print(12, x.size())

        return x, trans64x64

model = PoinNet(input_dim=6, num_classes=2).to(device)

In [7]:
# TEST MODEL
# points, _ = next(iter(train_dataloader))
# points = points.to(device)
# out, trans = model(points)

In [8]:
class CustomLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.cross_entropy_loss = nn.CrossEntropyLoss()

    def forward(self, inputs, targets, trans):
        d = trans.size(1)
        I = torch.eye(d).unsqueeze(0).to(device)
        loss = torch.linalg.norm(I - torch.bmm(trans, trans.transpose(2,1)), dim=(1,2))
        print(loss.size())
        loss = torch.mean(loss)
        return self.cross_entropy_loss(inputs, targets) + loss

In [9]:
for name, module in model.named_modules():
    if name == 'mlp8.1':
        print(name)
    # print(name)

mlp8.1


In [10]:
def train_PointNet(num_epochs, pointnet, train_dataloader, device, num_classes):
    pointnet.train()

    # def add_hooks(model):
    #     def hook(module, input, output):
    #         writer.add_histogram(f'{name}_outputs', output.detach().cpu().numpy(), epoch*3+batch_idx)

    #     for name, module in model.named_modules():
    #         module.register_forward_hook(hook)
    # add_hooks(pointnet)

    loss_func = CustomLoss()
    optimizer = optim.Adam(pointnet.parameters(), lr = 0.01)

    for epoch in range(num_epochs):
        for batch_idx, (points, labels) in enumerate(train_dataloader):
            points, labels = points.to(device), labels.to(device)

            optimizer.zero_grad()
            out, trans = pointnet(points)
            print('OUT',out.size())
            out = out.view(-1, num_classes)
            print('RESHAPED',out.size())
            labels = labels.view(-1)
            loss = loss_func(out, labels, trans)
            # writer.add_scalar("Loss/train", loss, epoch)
            loss.backward()
            optimizer.step()
 
            print(f"Epoch: {epoch+1}\tBatch ID: {batch_idx}\tLoss: {loss.item()}")
            print()

            # for name, param in pointnet.named_parameters():
            #     writer.add_histogram(name, param.clone().cpu().data.numpy(), epoch*3+batch_idx)
            

train_PointNet(5, model, train_dataloader, device, num_classes=2)
# writer.flush()
# writer.close()

4 torch.Size([64, 128, 5000])
5 torch.Size([64, 1024, 5000])
6 torch.Size([64, 1024])
7 torch.Size([64, 1024, 5000])
8 torch.Size([64, 1088, 5000])
9 torch.Size([64, 5000, 512])
10 torch.Size([64, 5000, 256])
11 torch.Size([64, 5000, 128])
12 torch.Size([64, 5000, 2])
OUT torch.Size([64, 5000, 2])
RESHAPED torch.Size([320000, 2])
torch.Size([64])
Epoch: 1	Batch ID: 0	Loss: 109.40735626220703

4 torch.Size([64, 128, 5000])
5 torch.Size([64, 1024, 5000])
6 torch.Size([64, 1024])
7 torch.Size([64, 1024, 5000])
8 torch.Size([64, 1088, 5000])
9 torch.Size([64, 5000, 512])
10 torch.Size([64, 5000, 256])
11 torch.Size([64, 5000, 128])
12 torch.Size([64, 5000, 2])
OUT torch.Size([64, 5000, 2])
RESHAPED torch.Size([320000, 2])
torch.Size([64])
Epoch: 1	Batch ID: 1	Loss: 88.26679229736328

4 torch.Size([22, 128, 5000])
5 torch.Size([22, 1024, 5000])
6 torch.Size([22, 1024])
7 torch.Size([22, 1024, 5000])
8 torch.Size([22, 1088, 5000])
9 torch.Size([22, 5000, 512])
10 torch.Size([22, 5000, 256])


In [14]:
from sklearn.metrics import f1_score
outputs = None
points = None
x = y = None
def test_PointNet(pointnet, test_dataloader, device):
    global outputs, points, x, y
    pointnet.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for idx, (points, labels) in enumerate(test_dataloader):
            points, labels = points.to(device), labels.to(device)
            outputs, _ = pointnet(points) # DO LOG SOFTMAX HERE
            outputs = F.log_softmax(outputs)
            print(outputs.size())
            print('checkpoint', outputs[0, 0])
            _, predicted = torch.max(outputs.data, 2)
            print(predicted.size())
            total += labels.size(0) * labels.size(1)
            correct += (predicted == labels).sum().item()
            # print(points.transpose(1,2)[0][:10].size())
            # print(labels.size())
            print(np.hstack([points.transpose(1,2)[0][:10].to('cpu'), labels[:10].to('cpu'), (predicted[:10]==labels[:10]).to('cpu')])[:20])
            x = labels.view(-1).to('cpu').numpy()
            y = predicted.view(-1).to('cpu').numpy()
            print(sum(x != y))
            print(sum(y == 1))
            f1 = f1_score(x, y)
            print('F1 score: ', f1)
            # break
    
test_PointNet(model, test_dataloader, device)

4 torch.Size([64, 128, 5000])
5 torch.Size([64, 1024, 5000])
6 torch.Size([64, 1024])
7 torch.Size([64, 1024, 5000])
8 torch.Size([64, 1088, 5000])
9 torch.Size([64, 5000, 512])
10 torch.Size([64, 5000, 256])
11 torch.Size([64, 5000, 128])
12 torch.Size([64, 5000, 2])
torch.Size([64, 5000, 2])
checkpoint 

C:\Users\and13375\AppData\Local\Temp\1\ipykernel_16516\597524401.py:14: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  outputs = F.log_softmax(outputs)


tensor([-4.1589, -4.1589], device='cuda:0')
torch.Size([64, 5000])
[[0.004283   0.4452669  0.0191022  ... 0.         0.         0.        ]
 [0.004283   0.44536626 0.01846546 ... 0.         1.         1.        ]
 [0.004283   0.44539109 0.01942057 ... 0.         0.         0.        ]
 ...
 [0.00430977 0.44561464 0.01942057 ... 0.         0.         0.        ]
 [0.00430977 0.44566432 0.0191022  ... 0.         1.         0.        ]
 [0.00430977 0.445714   0.01846546 ... 0.         0.         0.        ]]
180074
169801
F1 score:  0.15034585586350727
4 torch.Size([18, 128, 5000])
5 torch.Size([18, 1024, 5000])
6 torch.Size([18, 1024])
7 torch.Size([18, 1024, 5000])
8 torch.Size([18, 1088, 5000])
9 torch.Size([18, 5000, 512])
10 torch.Size([18, 5000, 256])
11 torch.Size([18, 5000, 128])
12 torch.Size([18, 5000, 2])
torch.Size([18, 5000, 2])
checkpoint 

C:\Users\and13375\AppData\Local\Temp\1\ipykernel_16516\597524401.py:14: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  outputs = F.log_softmax(outputs)


tensor([-2.8904, -2.8904], device='cuda:0')
torch.Size([18, 5000])
[[0.48684317 0.24039346 0.32250875 ... 1.         1.         1.        ]
 [0.48676285 0.23987183 0.34352118 ... 1.         1.         1.        ]
 [0.48689669 0.23823245 0.30945557 ... 0.         1.         0.        ]
 ...
 [0.4875927  0.23102909 0.14517669 ... 0.         0.         1.        ]
 [0.4875927  0.23003551 0.14390321 ... 0.         0.         1.        ]
 [0.4875927  0.22911647 0.14453995 ... 0.         0.         0.        ]]
49863
48662
F1 score:  0.17922338726934536


In [12]:
print(sum(x))
print(sum(y))
print(outputs.size())

12089
48662
torch.Size([18, 5000, 2])


In [13]:
print(f"Labels has {sum(x == 0)} zeroes and {sum(x == 1)} ones")
print(f"Predictions has {sum(y == 0)} zeroes and {sum(y == 1)} ones")


# writer.add_graph(model, points)
# writer.flush()
# writer.close()

Labels has 77911 zeroes and 12089 ones
Predictions has 41338 zeroes and 48662 ones


73.4029268292683% on roofs (5)
86.77414634146342% on trees (8)